In [ ]:
# I have a tendency to import everthing on the planet so I hope u have ram

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras_tuner as kt
import keras_cv as kcv

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras.regularizers import l1, l2, L1L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tifffile as tiff
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import imagecodecs
import cv2
import os
import PIL

from sklearn.utils.class_weight import compute_class_weight

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import seaborn as sns


In [ ]:
df = pd.read_excel('Path1/Path1-Model Training/Path1 Challenge Training Data.xlsx')
directory = 'Path1/Path1-Model Training/Path1 Challenge Training Images/'

In [ ]:
def load_images_from_folder(folder, target_size=(224, 224)):
    images = []
    files = sorted(os.listdir(folder))
    for filename in files:
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            resized_img = cv2.resize(img, target_size)
            images.append(resized_img)
    return np.array(images)

In [ ]:
directory = 'Path1/Path1-Model Training/Path1 Challenge Training Images/'

df = pd.read_excel('Path1/Path1-Model Training/Path1 Challenge Training Data.xlsx')
df = df[df['Grade Category'] != 'Standard']

grade_mapping = {'Select': 0, 'Low Choice': 1, 'Upper 2/3 Choice': 2, 'Prime': 3}
class_names = "Select", "Low Choice", "Upper 2/3 Choice", "Prime"

df['Label'] = df['Grade Category'].map(grade_mapping)

In [ ]:
file_list = sorted(os.listdir(directory))
df_filtered = df[df['Filename'].isin(file_list)]

In [ ]:
df_filtered.shape

In [ ]:
all_images = load_images_from_folder(directory, target_size=(224, 224))

all_labels = []
filtered_images = []

# extract label from each filename
for filename in file_list:
    if filename in df_filtered['Filename'].values:
        label = df_filtered[df_filtered['Filename'] == filename]['Label'].values[0]
        all_labels.append(label)

        #filter images because the first one file is not in the filtered df
        filtered_images.append(all_images[file_list.index(filename)])

# make them arrays
all_labels = np.array(all_labels)
filtered_images = np.array(filtered_images)

In [ ]:
train_data, validation_data = train_test_split(df_filtered, test_size=0.2, random_state=42)

# split the images themselves in case we do some visualizations
train_images, validation_images, train_labels, validation_labels = train_test_split(
    filtered_images,
    all_labels,
    test_size=0.2,
    random_state=42
)

datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
print(df_filtered.tail())
print(train_data['Label'].unique())
print(validation_data['Label'].unique())

In [ ]:
# one hot encoding allows for ml to digest categorical data as binary
train_labels_one_hot = to_categorical(train_data['Label'], num_classes=4)
validation_labels_one_hot = to_categorical(validation_data['Label'], num_classes=4)

In [ ]:
#oversampling... not the greatest

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(filtered_images.reshape(-1, 224*224*3), all_labels)

X_resampled = X_resampled.reshape(-1, 224, 224, 3)
y_resampled_one_hot = to_categorical(y_resampled, num_classes=4)


In [ ]:
# under sampling
sampling_strategy = {
    1: 200,
    2: 200,
    0: 175,
    3: 79
}

rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = rus.fit_resample(filtered_images.reshape(-1, 224*224*3), all_labels)

X_resampled = X_resampled.reshape(-1, 224, 224, 3)
y_resampled_one_hot = to_categorical(y_resampled, num_classes=4)


In [ ]:


#augmentation
datagen = ImageDataGenerator(
    rescale=1./255, #normalize!
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest' #brendan said he likes this one
)

# inputs our images into the augment and preps into tensors for model
train_generator = datagen.flow(
    # X_resampled,
    # y_resampled_one_hot,
    all_images[train_data.index],
    train_labels_one_hot,
    batch_size=32,
    shuffle=True
)

# preps images for model
validation_generator = datagen.flow(
    all_images[validation_data.index],
    validation_labels_one_hot,
    batch_size=32,
    shuffle=False
)

In [ ]:
image_filename = df['Filename'].iloc[0]  # first image

image_path = os.path.join(directory, image_filename)

# plot
img = image.load_img(image_path, target_size=(224, 224))
plt.imshow(img)
plt.axis('off')
plt.title(f"Image: {image_filename}")
plt.show()

In [ ]:
# check  original balance
class_counts = df_filtered['Grade Category'].value_counts()
print(class_counts)


In [ ]:
#post under/oversampling

unique_classes, counts = np.unique(y_resampled, return_counts=True)
for cls, count in zip(unique_classes, counts):
    print(f"Class {cls}: {count} samples")

In [ ]:

# Calculate class frequencies
unique_classes, class_counts = np.unique(y_resampled, return_counts=True)

# Calculate class weights
total_samples = len(y_resampled)
class_weights = {cls: total_samples / (len(unique_classes) * count) for cls, count in zip(unique_classes, class_counts)}

print(class_weights)

In [ ]:
class_weights_dict = {
    1: 0.5,
    2: 1,
    0: 1,
    3: 1.5
}

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = Sequential()

model.add(base_model)

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax'))


In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss=kcv.losses.FocalLoss(), metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# TTRAINNNN
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    class_weight=class_weights_dict,
    callbacks=[early_stopping]
)

# eval on validation
loss, accuracy = model.evaluate(validation_generator, verbose=1)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")



In [ ]:
print(f"Validation Accuracy: {accuracy * 100:.2f}%")